In [200]:
import requests
import re,os
from time import sleep
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
from pprint import pprint
from tqdm.notebook import tqdm
import mojimoji

example_url = 'http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuroku/contents/2154.html'

In [164]:
def editor(dd_tag):
    try:
        editor_name = dd_tag.text.split('\n')[1].replace('\u3000','')
    except:
        editor_name = 'No name'
    return editor_name

def title(dd_tag):
    try:
        title_list = list(dd_tag.a.childGenerator())
        english_title = title_list[0]#.replace('\n',' ')
        japanese_title = title_list[2]
        return english_title,japanese_title
    except:
        try:
            return dd_tag.a.text,''
        except:
            return '',''

def link(dd_tag,kokyuroku_path='http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuroku/'):
    try:
        path = kokyuroku_path+dd_tag.a.get('href')
    except:
        path = 'No path'
    return path

In [196]:
def scrape_koukyuroku_list(start_year=1964,end_year=2020,print_info=False):
    data = []
    for year in tqdm(range(start_year,end_year+1)):
        res = requests.get('http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuroku/{0}.html'.format(year))
        soup = bs(res.text.encode(res.encoding),'html5lib')
        dd = soup.find_all('dd')
        numbers = reversed(list(map(lambda x:x.text[3:],soup.find_all('dt'))))
        editors = reversed(list(map(editor, dd)))
        titles = reversed(list(map(title, dd)))
        links = reversed(list(map(link, dd)))
        for nu,au,ti, li in zip(numbers,editors,titles,links):
            data.append([nu,au,ti[0],ti[1],li,year])
            if print_info:
                print(nu,au,ti[0],ti[1])
        sleep(1)
    
    # DataFrameへの変換
    col = ['editors','English Title','Japanese Title','Link','Year']
    data_np = np.array(data)
    for i in range(data_np.shape[0]):
        for j in range(data_np.shape[1]):
            if '\n' in data_np[i,j] or '\r' in data_np[i,j]:
                data_np[i,j] = data_np[i,j].replace('\n','').replace('\r','')
    data_df = pd.DataFrame(data_np[:,1:],index=data_np[:,0],columns=col)
    return data_df

def save_koukyuroku_list(data_df,save_path):
    data_df.to_csv(save_path)
    
def remove_new_lines(csv_path,save_path):
    #改行消す
    with open(csv_path,'r') as f:
        lines = f.readlines()
        new_lines = [lines[0]]
        for i in range(1,len(lines)):
            line = lines[i]
            if re.fullmatch(r'[0-9]+',line.split(',')[0]):
                new_lines.append(line)
            else:
                new_lines[-1] = ''.join([new_lines[-1][:-1],line])
    
    #全角英字を消してeditorがいないものは'No editor'にする
    for i in range(len(new_lines)):
        record = mojimoji.zen_to_han(new_lines[i].replace('，','・'), kana=False, ascii=True).split(',')
        if record[1].replace(' ','')==(''.join(record[2:4]).strip().replace(' ','')):
            record[1] = 'No editor'
        if re.fullmatch(r"[a-zA-Z0-9\(\), -:']+",record[2]) is None and record[3]=='':
            record[2],record[3] = record[3], record[2]
        if re.fullmatch(r"[a-zA-Z0-9\(\), -:']+",record[3]) and record[2]=='':
            record[2],record[3] = record[3], record[2]
        new_lines[i] = ",".join(record)
        
    with open(save_path, 'w') as f:
        f.write(''.join(new_lines))
        f.flush()
        
def pipeline_scrape_to_save(start_year=1964,end_year=2020):
    csv_path = f'./kokyuroku_{start_year}_to_{end_year}.csv'
    data_df = scrape_koukyuroku_list(start_year,end_year)
    save_koukyuroku_list(data_df,csv_path)
    remove_new_lines(csv_path,csv_path)

In [168]:
pipeline_scrape_to_save()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


A Jupyter Widget

In [12]:
# res = requests.get('http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuroku/2008.html')
# soup = bs(res.text.encode(res.encoding),'html5lib')
# dd = soup.find_all('dd')
# pprint(list(zip(list(map(lambda x:x.text[3:], soup.find_all('dt'))),list(map(title, dd)),list(map(author, dd)),list(map(link, dd)))))
# pprint(list(map(title, dd)))
# pprint(list(map(link, dd)))
# pprint(list(map(title, dd)))

In [ ]:
# data = []
# for year in tqdm(range(1964,2021)):
#     res = requests.get('http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuroku/{0}.html'.format(year))
#     soup = bs(res.text.encode(res.encoding),'html5lib')
#     dd = soup.find_all('dd')
#     numbers = reversed(list(map(lambda x:x.text[3:],soup.find_all('dt'))))
#     authors = reversed(list(map(author, dd)))
#     titles = reversed(list(map(title, dd)))
#     links = reversed(list(map(link, dd)))
#     for nu,au,ti, li in zip(numbers,authors,titles,links):
#         data.append([nu,au,ti[0],ti[1],li,year])
#         print(nu,au,ti[0],ti[1])
#     sleep(1)

In [253]:
# data_np = np.array(data)
# for i in range(data_np.shape[0]):
#     for j in range(data_np.shape[1]):
#         if '\n' in data_np[i,j] or '\r' in data_np[i,j]:
#             print(repr(data_np[i,j]))
#             data_np[i,j] = data_np[i,j].replace('\n','').replace('\r','')
# col = ['authors','English Title','Japanese Title','Link','Year'
# df = pd.DataFrame(data_np[:,1:],index=data_np[:,0],columns=col)
# df

In [255]:
# df.to_csv('./kokyuroku.csv')

In [8]:
# pd.read_csv('./kokyuroku.csv',index_col=0)

In [10]:
# with open('./kokyuroku.csv') as f:
#     lines = f.readlines()
#     new_lines = [lines[0]]
#     for i in range(1,len(lines)):
#         line = lines[i]
#         if re.fullmatch(r'[0-9]+',line.split(',')[0]):
#             new_lines.append(line)
#         else:
#             new_lines[-1] = ''.join([new_lines[-1][:-1],line])

# for i in range(len(new_lines)):
#     record = mojimoji.zen_to_han(new_lines[i].replace('，','・'), kana=False, ascii=True).split(',')
#     if record[1].replace(' ','')==(''.join(record[2:4]).strip().replace(' ','')):
#         record[1] = 'No author'
#     new_lines[i] = ",".join(record)

# with open('./kokyuroku_organized.csv', 'w') as f:
#     f.write(''.join(new_lines))
#     f.flush()

In [11]:
pd.read_csv('./kokyuroku_organized.csv',index_col=0)

,authors,English Title,Japanese Title,Link,Year
1,"中野茂男 (NAKANO,SHIGEO )",Some Analytic Structures Associated to Algebra...,代数曲線に附随する二三の解析的構造,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1964
2,岩野 正宏、木村 俊房、大久保謙二郎,境界層と変わり点に関するシンポジウム報告(I),NaN,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1964
3,角谷 典彦、今井 巧、森口 治生,境界層と変わり点に関するシンポジウム報告(II),NaN,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1964
4,"阿曾義之 (ASO,YOSHIYUKI )",Numerical Solution of Singular Integral Equati...,特異積分方程式の数値解法,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1965
5,No author,Proceeding of the Conference on Operator Ring ...,作用素環とその物理的応用に関する研究会報告集,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1965
6,No author,Proceeding of the Conference on the Mathematic...,散乱の理論の数学に関する研究会報告集,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1965
7,No author,Reports on the Symposium of the Numerical Comp...,流体力学における数値計算シンポジウム報告,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1965
8,No author,Reports on the Symposium of the Functional Equ...,最適制御問題の函数方程式研究シンポジウム報告1,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1966
9,"荒木不二洋 (ARAKI,HUZIHIRO )",Proceeding of the Conference on Operator Ring,作用素環研究会報告集,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1966
10,No author,Reports on the Symposium of the Numerical Comp...,流体力学における数値計算シンポジウム報告,http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuro...,1966


In [ ]:
res = requests.get()
soup = bs(res.text.encode(res.encoding),'html5lib')

In [213]:
def number_of_pdf(div_tag):
    return int(re.search('[0-9]+\.',div_tag.text).group(0).replace('.',''))
    
def title_of_pdf(div_tag):
    if div_tag.a is None:
        text = div_tag.text.split(' ')[1]
        text = text.split('-')[0]
        return text
    return div_tag.a.text
    
def link_of_pdf(div_tag):
    if div_tag.a is None: return ''
    RIMS_path = "http://www.kurims.kyoto-u.ac.jp/~kyodo/kokyuroku/contents/"
    pdf_path = div_tag.a.get('href').replace('./',RIMS_path)
    return pdf_path

def authors_of_pdf(div_tag):
    text = div_tag.text.strip()
    if '\u3000\u3000\u3000' in text:
        affiliations,authors = text.split('\u3000\u3000\u3000')
        affiliations = affiliations.split('/')
        affiliations = list(map(lambda x:x.strip(),affiliations))
    else: 
        authors = text.strip()
        
    if re.search(r'\([a-zA-Z]+',authors):
        try:
            authors_JPN,authors_ENG =  authors.split('(')
        except:
            start = re.search(r'\([a-zA-Z]+',authors).start()
            authors_JPN,authors_ENG =  authors[:start],authors[start+1:]
        authors_JPN = list(map(lambda x:x.strip(),authors_JPN.strip().split('/')))
        authors_ENG = list(map(lambda x:x.strip(),authors_ENG.replace(')','').strip().split('/')))
    else:
        authors_ENG = list(map(lambda x:x.strip(),authors.strip().split('/')))
        authors_JPN = ['']*len(authors_ENG)
    
    if not '\u3000\u3000\u3000' in text:
        affiliations = ['']*len(authors_ENG)
    return affiliations,authors_JPN,authors_ENG

def make_list_of_pdf_info(url):
    res = requests.get(url)
    soup = bs(res.text.encode(res.encoding),'html5lib')
    all_div = soup.find_all('div')
    
    i = 0
    list_pdf_info = []
    while i < len(all_div):
        div_tag = all_div[i]
        if re.search('[0-9]+\.',div_tag.text):
            number = number_of_pdf(div_tag)
            title = title_of_pdf(div_tag)
            link = link_of_pdf(div_tag)
            affiliations,authors_JPN,authors_ENG = authors_of_pdf(all_div[i+1])
            list_pdf_info.append([number,title,link,affiliations,authors_JPN,authors_ENG])
            i += 2
        else:
            i += 1
    return list_pdf_info

def save_pdf_info(url,kokyuroku_number):
    info = make_list_of_pdf_info(url)
    info = np.array([list(map(lambda x: ','.join(x) if isinstance(x,list) else x, l)) for l in info])
    columns = ['No.','title','link','Affiliation','Author in JPN','Author in ENG']
    info = pd.DataFrame(info,columns=columns).set_index('No.')
    info.to_csv(f'./pdf_info/{kokyuroku_number}.csv')
    
def save_all_pdf_info(kokyuroku_csv_path, print_title=False, skip_existing=True):
    df = pd.read_csv(kokyuroku_csv_path,index_col=0)
    kokyuroku_numbers = df.index.values
    links = df.values[:,3]
    titles = df.values[:,2]
    if os.path.exists('./error_log.txt'): os.remove('./error_log.txt')
    for i in tqdm(range(len(kokyuroku_numbers))):
        if print_title: print(titles[i]) if isinstance(titles[i],str) or not np.isnan(titles[i]) else print(df.values[i,1])
        if skip_existing and os.path.exists(f'./pdf_info/{kokyuroku_numbers[i]}.csv'): continue

        try:
            save_pdf_info(links[i],kokyuroku_numbers[i])
        except Exception as e:
            with open('./error_log.txt','a') as f:
                f.write(','.join([str(kokyuroku_numbers[i]),str(links[i]),str(e),'\n']))
                f.flush()
        finally:
            sleep(1)

In [214]:
kokyuroku_csv_path = './kokyuroku_1964_to_2020.csv'
save_all_pdf_info(kokyuroku_csv_path,print_title=True,skip_existing=True)

A Jupyter Widget

代数曲線に附随する二三の解析的構造 
境界層と変わり点に関するシンポジウム報告(I) 
境界層と変わり点に関するシンポジウム報告(II) 
特異積分方程式の数値解法 
作用素環とその物理的応用に関する研究会報告集 
散乱の理論の数学に関する研究会報告集 
流体力学における数値計算シンポジウム報告 
最適制御問題の函数方程式研究シンポジウム報告1 
作用素環研究会報告集 
流体力学における数値計算シンポジウム報告 
最適制御問題の函数方程式研究シンポジウム報告2 
数値解析セミナー報告1 
複素解析とその応用研究会報告1 
複素解析とその応用研究会報告2 
散乱の理論の数学に関する研究会報告集 
制御問題の解析的研究シンポジウム報告 
数値解析セミナー報告2 
数値解析の基礎理論および偏微分方程式の数値解法シンポジウム 
作用素環研究会報告集 
定常過程研究会報告集 
バンドルにおける位相幾何学的方法研究会報告集 
位相解析的方法による偏微分方程式論研究会及び散乱理論の数学研究会 報告集 
乱流の分布汎函数方程式の研究会報告集 
流体力学における非線型問題研究会報告集 
実験計画法研究会報告集 
倉持型理想境界と解析学 
統計的決定函数のAdmissibilityの研究報告集 
動的計画法研究会報告集 
複素多様体の変形─研究グループ報告─ 
光学スペクトルの特異点の研究会報告集 
準群の代数的理論 
発展方程式とその数値解析研究会報告集 
制御問題の数学的研究報告集 
微分方程式の数値解法研究会報告集 
最大値原理と半群の生成 
近似理論の研究報告集 
偏微分方程式の数値解法研究会報告集 
常微分方程式及び函数微分方程式研究会報告集 
近似理論の研究報告集 
位相解析的方法による偏微分方程式論研究会報告集 
代数的整数論研究会報告集─追加も含む(41─2)− 
数値解析の基礎理論研究会報告集 
輻射流体力学の運動方程式研究会報告集 
統計的多変量解析の研究報告集 
制御系方程式の研究報告集 
Invariant Statistical ProblemsとS ufficiency 
乱流の分布汎函数方程式研究会報告集 
非線型及び線型制御研究会報告集 
作用素環研究会報告集 
ホモトピー論研究会報告集 
近似計算とシミュレーションによる近似解法研究会報告集 
Navi

完全WKB解析と複素フーリエ解析
保型形式とゼータ関数の研究
作用素環における双加群と量子群の研究
線型行列不等式と半正定値計画法
Discretizations of Integrable Systems: Theory and Applications
実特異点のトポロジーとその関連話題
非正則推定論の構造の研究
群と等質空間の表現論
非線形発展方程式とその応用
非古典論理とKripke意味論の新局面
疑似乱数とカオス
Convolutionの新しい展開
量子確率論とエントロピー解析
漸近解析に於る幾何学的方法
離散数理と連続数理における最適化理論
ポテンシャル論とその関連分野
無限次元測度論と無限次元群の表現論
矩形管流れの解の構造
数学史の研究
離散可積分系と離散解析
sequent calculusとKripke semanticsに基づく非古典論理学の研究
Analysis of Discrete Groups II
Type Theoryとそのコンピュータシステムへの応用
作用素環論の深化
変分問題とその周辺
代数的整数論とその周辺
作用素論における不等式とその周辺
短期共同 複素領域の偏微分方程式
流れの非線形性と乱流の統計性質
波動現象におけるパターンの生成と特異性
非線形解析学と凸解析学の研究
短期共同 確率数値解析に於ける諸問題, III
特異点と複素解析幾何
短期共同 非線形の数理と関数方程式
短期共同 無限次元非可換解析学の動向
偏微分方程式の解の構造の研究
短期共同 CR geometryと孤立特異点
数式処理における理論と応用の研究
応用函数解析の研究
数値計算アルゴリズムの研究
アルゴリズムと計算の理論
複素力学系の諸問題
決定理論とその関連分野
等質空間と部分多様体の幾何学
短期共同 非線形発展方程式の解の正則性と解の爆発との関連
短期共同 Exact C*-環とその周辺
スペクトル・散乱理論とその周辺
短期共同 不確実な環境モデルでの動的行動決定システム
調和・解析関数空間と線形作用素
局所的及び大域的特異点論の研究
渦度場のダイナミックスと乱流の数理
保型形式と整数論
短期共同 6j-symbolから導かれる位相的量子場の理論の研究
短期共同 離散的アルゴリズムと計算量
短期共同 Large Deviation and Stat

部分多様体の微分幾何学
最適化数理の手法と実際
確率数値解析に於ける諸問題, VII
紀要の電子化と周辺の話題
変分問題とその周辺
組合せデザインとその周辺における数理的基礎およびそれらの応用
有限群のコホモロジー論とその周辺
群の表現と調和解析の広がり
保型形式と保型L函数
自然数の超準モデルにおける1階定義可能性の研究
単葉関数論における坂口関数とその応用
強制法と無限組合せ論
複雑流体の数理とその応用
ソリトン理論から可積分数理へ:“de nouvelles perspectives "
関数方程式の解のダイナミクスと数値シミュレーション
現象の数理モデルと発展方程式
代数的組合せ論とその周辺
不確実性の下での意思決定と数理モデル
可換 Banach 環と種々の分野との交流
スペクトル・散乱理論とその周辺
可解格子模型の最近の進展
微分方程式の粘性解理論とその発展
繰り込み群の数理科学での応用
非線形波動の数理と応用
非線形解析学と凸解析学の研究
力学系の研究 ― トポロジーと計算機による新展開
作用素空間論とその応用について
ベルグマン核の解析幾何とその周辺
経済の数理解析
計算理論とアルゴリズムの新展開
代数幾何と位相幾何の周辺
調和解析と非線形偏微分方程式
一般・幾何学的トポロジーと幾何学的群論
複素漸近解析の新展開と応用 ― 力学系, 総和法, 連分数をめぐって ―
複素力学系とその周辺分野の研究
流体と気体の数学解析
混合、化学反応、燃焼の流体力学
組合せ論的表現論の世界
反応拡散系に現れる時・空間パターンのメカニズム
第2回生物数学の理論とその応用
力学系と微分幾何学
Recent Topics on Real and Complex Singularities
カルタン幾何の進化発展とそれに関連する数学の諸問題
代数、言語、計算システムにおけるアルゴリズム問題
情報決定過程論の展開
計算科学の基盤技術とその発展
統計的条件付推測とそれに関連する話題
量子解析におけるミクロ・マクロ双対性
Capelli 恒等式の新局面
積分核の代数解析的研究
スペクトル・散乱理論とその周辺
解析的整数論とその周辺
解析的整数論
数学史の研究
Computer Algebra - Design of Algorithms, Implementations and A

数学と生命現象の連関性の探究 〜新しいモデリングの数理〜
モデル理論における独立概念と次元の研究
デザイン、符号、グラフおよびその周辺
非線形波動現象の数理と応用
スペクトル・散乱理論とその周辺
抽象的発展方程式の新たなる役割 - 個々の偏微分方程式を俯瞰する観点から -
作用素単調関数と関連する話題について
計算理論とアルゴリズムの新潮流
反映原理と巨大基数の集合論
偏微分方程式の解の幾何
Fano 多様体の最近の進展
解析的整数論−超越関数の数論的性質とその応用
マクロ経済動学と非線形数理
生物流体力学における流れ構造の解析と役割
常微分方程式の定性的理論の新展開
スペクトル・散乱理論とその周辺
確率論シンポジウム
繰りこみ群の数理科学での応用
非圧縮性粘性流体の数理解析
不確実さと曖昧さの数理
数式処理とその周辺分野の研究
高次元量子トモグラフィにおける統計理論的なアプローチ
数学ソフトウェアとその効果的教育利用に関する研究
Asymptotic Statistics and Its Related Topics
Intelligence of Low-dimensional Topology
不確実性の下での数理的意思決定の理論と応用
ヤング図形・統計物理に関連する代数的組合せ論
保存則をもつ偏微分方程式に対する解の正則性・特異性の研究
計算機科学における論理・代数・言語
統計多様体の幾何学の新展開
第10回生物数学の理論とその応用
ミラー対称性の展望
偏微分方程式の背後にある確率過程と解の族が示す統計力学的な現象の解析
数学教師に必要な数学能力の育成法に関する研究
量子場の数理とその周辺
変換群の位相幾何と代数構造
非線形解析学と凸解析学の研究
非線形現象に現れるパターン形成の数理解析
表現論と調和解析の新たな進展
有限群とその表現,  頂点作用素代数,  代数的組合せ論の研究
数式処理研究の新たな発展
ウェーブレット解析とサンプリング理論
群作用と部分多様体論の展開
数式処理研究の新たな発展
最適化アルゴリズムの進展:理論・応用・実装
集合論的・幾何学的トポロジーと種々の分野の交流
ファイナンスの数理解析とその応用
保型形式および関連するゼータ関数の研究
作用素の平均とその関連
離散群と双曲空間の複素解析とトポロジー
第11回生物数学の理論とその応用
モ

In [206]:
isinstance(np.nan,np.float)

True

In [203]:
type(np.nan)

float